In [46]:
#!pip install gradio

In [25]:
import ollama
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [2]:
# 환경 변수 로드
from dotenv import load_dotenv
import os
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_ANON_KEY = os.getenv("SUPABASE_ANON_KEY")

In [3]:
# 수파베이스 접속
import supabase
sb = supabase.create_client(SUPABASE_URL, SUPABASE_ANON_KEY)

In [6]:
# 데이터 가져오기
scraps = sb.table('scraps').select("*").execute().data

In [10]:
scraps_df = pd.DataFrame(scraps)
scraps_df.head()


,id,created_at,title,description,url,address,image
0,69157,2024-04-27T03:04:49.787907+00:00,"화곡 러브레터, 강서·화곡·까치산역·목동, 서울 - 여기어때 특가",러브레터는 현장결제 불가입니다 어플로 예약부탁드립니다\n\n인원추가는 불가능 합니다,https://www.yeogi.com/domestic-accommodations/...,서울 강서구 화곡동 1116-11,https://www.yeogi.com/_next/image?url=https%3A...
1,70163,2024-04-27T03:04:49.928672+00:00,"서울신라호텔, 을지로·시청·명동, 서울 - 여기어때 특가","전통이라는 지붕 위에 모더니즘적 디자인 요소를 가미, 삶에 여유와 품격을 한층 높여...",https://www.yeogi.com/domestic-accommodations/...,서울 중구 장충동2가 202,https://www.yeogi.com/_next/image?url=https%3A...
2,54719,2024-04-27T03:04:49.984391+00:00,"서울 나나하우스, 영등포·신림·김포공항, 서울·경인 - 여기어때 특가",안녕하세요~ 나나입니다:)\n\n나나하우스는 김포 공항에서 걸어서 15 분(택시 기...,https://www.yeogi.com/domestic-accommodations/...,서울 강서구 공항동 24-4 4층 나나하우스,https://www.yeogi.com/_next/image?url=https%3A...
3,5629,2024-04-27T03:04:50.043968+00:00,"신림 URBAN HOTEL&GOLF, 서울대·신림·사당·동작, 서울 - 여기어때 특가",HOTEL 어반은 원칙을 지킵니다.\n도심 속 Boutique & Business ...,https://www.yeogi.com/domestic-accommodations/...,서울 관악구 신림동 1569-5,https://www.yeogi.com/_next/image?url=https%3A...
4,6278,2024-04-27T03:04:50.10257+00:00,"★당일특가★ 서울가든호텔, 마포·홍대·신촌·서대문, 서울 - 여기어때 특가",서울가든호텔은 40년 이상의 호텔 경영 노하우와 서비스 경험을 축적하여 비즈니스 고...,https://www.yeogi.com/domestic-accommodations/...,,


In [16]:
# 데이터 전체 임베딩 벡터 추출
scraps_df['embedding'] = scraps_df['title'].progress_apply(lambda x: ollama.embeddings(model="llama3", prompt=x.replace(', 서울 - 여기어때 특가', ''))['embedding'])

100%|██████████| 1000/1000 [01:49<00:00,  9.12it/s]


In [18]:
keyword_embedding = ollama.embeddings(model="llama3", prompt='호텔')
keyword_embedding

{'embedding': [1.0541527271270752,
  3.7456252574920654,
  0.1500774770975113,
  0.36355647444725037,
  -2.0378572940826416,
  0.3314909338951111,
  -0.8675970435142517,
  0.26895788311958313,
  -3.892367124557495,
  -1.4351105690002441,
  -0.12301095575094223,
  -0.5757513642311096,
  -1.4058763980865479,
  0.3269679844379425,
  -3.1396706104278564,
  2.98160719871521,
  -2.520904064178467,
  0.4052010476589203,
  -2.7407357692718506,
  2.6347897052764893,
  -0.48023176193237305,
  -0.26480525732040405,
  0.8256552815437317,
  0.024226181209087372,
  -0.07426367700099945,
  -2.9023025035858154,
  0.45901355147361755,
  0.6275045871734619,
  -0.32450544834136963,
  0.3209134042263031,
  -2.4958810806274414,
  -0.3210790753364563,
  0.340629905462265,
  0.14352461695671082,
  -0.6048107147216797,
  -0.01821625418961048,
  -1.093572735786438,
  -2.504976987838745,
  -0.027036158367991447,
  -1.5077240467071533,
  1.7037702798843384,
  0.9741539359092712,
  -0.5446135997772217,
  -1.39486

In [24]:
all_embeddings = scraps_df.embedding.tolist()



In [28]:
keyword_embedding = keyword_embedding['embedding']



In [29]:
cosine_similarity([keyword_embedding], all_embeddings)

array([[0.57822705, 0.64004348, 0.61690509, 0.24288339, 0.62912551,
        0.65111372, 0.66258459, 0.57675211, 0.64301263, 0.64903239,
        0.63035714, 0.64166909, 0.63894616, 0.55080885, 0.44843861,
        0.57742784, 0.67967449, 0.64729157, 0.6135677 , 0.55235395,
        0.64904385, 0.61523971, 0.58061628, 0.64910649, 0.60499391,
        0.59741723, 0.6598468 , 0.56625841, 0.61411153, 0.65349216,
        0.58498887, 0.63908417, 0.54499396, 0.62965094, 0.60208876,
        0.65224206, 0.62494495, 0.62731577, 0.56666316, 0.562138  ,
        0.3085691 , 0.58277826, 0.57675722, 0.61450572, 0.59093184,
        0.59019188, 0.6504149 , 0.65070821, 0.62988592, 0.60394809,
        0.6374669 , 0.64842407, 0.5583229 , 0.65034137, 0.41870555,
        0.48956249, 0.65267941, 0.6109955 , 0.58786304, 0.62537022,
        0.63473892, 0.63406744, 0.59015812, 0.52425034, 0.70367158,
        0.66605913, 0.59889236, 0.64517797, 0.56704226, 0.53632515,
        0.5752438 , 0.6178948 , 0.59365708, 0.61

In [32]:
def search_engine(vertor_df, keyword, top_n=5):
    # 키워드 임베딩 벡터 추출
    keyword_embedding = ollama.embeddings(model="llama3", prompt=keyword)['embedding']
    all_embeddings = vertor_df.embedding.tolist()
    
    # 코사인 유사도 계산
    similarities = cosine_similarity([keyword_embedding], all_embeddings)
    
    # 유사도를 기준으로 내림차순 정렬 후 상위 n개 선택
    top_indices = similarities.argsort()[0][::-1][:top_n]
    
    # 가장 유사한 항목 추출
    return vertor_df.iloc[top_indices]

In [39]:
search_engine(scraps_df, '수유')

,id,created_at,title,description,url,address,image,embedding
469,12500,2024-04-27T03:05:18.007485+00:00,"수유 솔리드, 강북·수유·미아, 서울 - 여기어때 특가","수유 솔리드 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,,,"[-1.0290662050247192, 1.3548228740692139, -0.7..."
189,45309,2024-04-27T03:05:01.014261+00:00,"★당일특가★ 신라스테이 서초, 서초·교대, 서울 - 여기어때 특가","강남역 인근에 있어 관광, 쇼핑, 비즈니스 모두에 적합한 호텔로서, 투숙객들의 편리...",https://www.yeogi.com/domestic-accommodations/...,,,"[0.4550277888774872, 1.0039235353469849, -1.47..."
787,4718,2024-04-27T03:05:38.072941+00:00,"수유 다니엘캄파넬라, 강북·수유·미아, 서울 - 여기어때 특가",디자인호텔 다니엘 캄파넬라는...\n강북도봉노원 “최초 & 최고의 디자인호텔”입니다...,https://www.yeogi.com/domestic-accommodations/...,,,"[-0.9276286959648132, 0.8271644711494446, -0.3..."
594,1968,2024-04-27T03:05:27.353232+00:00,"수유 리츠, 강북·수유·미아, 서울 - 여기어때 특가","수유 리츠 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,,,"[-0.8710508346557617, 0.8025059700012207, -0.6..."
990,3717,2024-04-27T03:05:51.24887+00:00,"신천 마쯔&2060, 잠실새내·신천, 서울 - 여기어때 특가","신천 마쯔&2060 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,서울 송파구 잠실동 183-6,https://www.yeogi.com/_next/image?url=https%3A...,"[-2.6874375343322754, 1.715651512145996, -0.05..."


In [40]:
search_engine(scraps_df, '신천')

,id,created_at,title,description,url,address,image,embedding
990,3717,2024-04-27T03:05:51.24887+00:00,"신천 마쯔&2060, 잠실새내·신천, 서울 - 여기어때 특가","신천 마쯔&2060 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,서울 송파구 잠실동 183-6,https://www.yeogi.com/_next/image?url=https%3A...,"[-2.6874375343322754, 1.715651512145996, -0.05..."
636,5470,2024-04-27T03:05:28.784956+00:00,"신천 라비앙, 잠실새내·신천, 서울 - 여기어때 특가","신천 라비앙 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,,,"[-0.6959884762763977, 1.5255078077316284, -0.1..."
64,1425,2024-04-27T03:04:53.703474+00:00,"신천 식스티, 잠실새내·신천, 서울 - 여기어때 특가",1.숙박손님 새벽5시까지 아무런 연락없이 입실하지\n\n않으면 노쇼로 간주하여 입실...,https://www.yeogi.com/domestic-accommodations/...,서울 송파구 잠실동 181-19,,"[-1.528183102607727, 0.8699812889099121, -0.74..."
625,52376,2024-04-27T03:05:27.805644+00:00,"신천 라몬, 잠실새내·신천, 서울 - 여기어때 특가","신천 라몬 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,,,"[-0.5626351237297058, 0.9340556263923645, -0.2..."
429,5861,2024-04-27T03:05:15.810921+00:00,"신천 WYNN-윈, 잠실새내·신천, 서울 - 여기어때 특가",-넷플릭스 이용가능(자동 로그인)\n\nPC에서 실행 후 리모컨 외부입력으로 TV와...,https://www.yeogi.com/domestic-accommodations/...,,,"[-2.3063180446624756, 1.3658162355422974, -2.4..."


In [43]:
search_engine(scraps_df, '분위기 좋은 곳')


,id,created_at,title,description,url,address,image,embedding
250,1962,2024-04-27T03:05:05.022336+00:00,"천호 무오래, 천호·길동·둔촌, 서울 - 여기어때 특가","천호 무오래 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,,,"[0.13487094640731812, -1.9811100959777832, -0...."
90,69308,2024-04-27T03:04:55.257638+00:00,"중구 요기는 무인텔, 용산·중구·명동·이태원·서울역, 서울 - 여기어때 특가","12월 24, 25, 31일 전화예약만 받습니다.",https://www.yeogi.com/domestic-accommodations/...,,,"[1.716164469718933, 0.32357892394065857, -1.90..."
993,67703,2024-04-27T03:05:51.432129+00:00,"길동 오라-ORA, 천호·길동·둔촌, 서울 - 여기어때 특가","길동 오라-ORA 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,서울 강동구 길동 412-14,https://www.yeogi.com/_next/image?url=https%3A...,"[0.5017898082733154, -1.9622070789337158, -1.0..."
189,45309,2024-04-27T03:05:01.014261+00:00,"★당일특가★ 신라스테이 서초, 서초·교대, 서울 - 여기어때 특가","강남역 인근에 있어 관광, 쇼핑, 비즈니스 모두에 적합한 호텔로서, 투숙객들의 편리...",https://www.yeogi.com/domestic-accommodations/...,,,"[0.4550277888774872, 1.0039235353469849, -1.47..."
384,51465,2024-04-27T03:05:12.867856+00:00,"길동 일루와, 천호·길동·둔촌, 서울 - 여기어때 특가","길동 일루와 위치와 시설, 사진과 후기를 확인하고 여기어때 특가로 예약하세요.",https://www.yeogi.com/domestic-accommodations/...,,,"[0.9962537884712219, -1.5956019163131714, -1.7..."


In [51]:
import gradio as gr

def gradio_search_interface(query):
    results = search_engine(scraps_df, query)
    # title과 image만 추출하여 리스트로 반환
    filtered_results = [result["title"] + "\n" + result["description"] for result in results.to_dict('records')][:5]
    return filtered_results[0], filtered_results[1], filtered_results[2], filtered_results[3], filtered_results[4]

iface = gr.Interface(
    fn=gradio_search_interface,
    inputs="text",
    outputs=[gr.Textbox(), gr.Textbox(), gr.Textbox(), gr.Textbox(), gr.Textbox()],
    title="시맨틱 검색 엔진",
    description="키워드를 입력하면 관련된 정보를 검색해줍니다."
)

iface.launch()


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


/Users/dante/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/analytics.py:99: UserWarning: unable to parse version details from package URL.
  warnings.warn("unable to parse version details from package URL.")
